# Sending email programmatically

In [1]:
# Import necessary libraries.
import requests
import json
import pandas as pd
import urllib.parse
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from tabulate import tabulate
from datetime import datetime

In [2]:
# 12 letter email app password must be added here.
password = ""
# Elastic Email API key must be added here.
api_key = ""

## Sending an email from gmail to gmail:

Check gmail through other email platforms: https://support.google.com/mail/answer/7126229?visit_id=638300612428115617-2230202100&p=BadCredentials&rd=2#cantsignin&zippy=%2Ci-cant-sign-in-to-my-email-client

Sign in with app password: https://support.google.com/accounts/answer/185833?sjid=13742701505310090574-NA

## Data Loading and Processing:

In [3]:
# Load data from JSON file
def load_data(file_path):
    """
    Load data from a JSON file.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        dict: A dictionary containing the loaded data.
    """
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

# Define a custom formatting function
def format_annual_wages(wage):
    """
    Format the annual wages value with commas as thousands separators.

    Parameters:
        wage (float): The annual wages value to be formatted.

    Returns:
        str: The formatted annual wages value as a string.

    Example:
        >>> format_annual_wages(50000)
        '50,000'
    """
    return "{:,.0f}".format(wage)

## Define hyperlinks

In [4]:
# Create hyperlinks
def create_hyperlink(row):
    """
    Create hyperlinks for the "Career" column based on provided parameters.

    Args:
        row (pandas.Series): A row of data from the DataFrame.

    Returns:
        str: An HTML hyperlink for the Career column.
    """
    base_url = "https://www.careeronestop.org/Toolkit/Careers/Occupations/occupation-profile.aspx?"
    parameters = {
        "keyword": urllib.parse.quote(row["Career"]),
        "location": "US",
        "lang": "en",
        "onetCode": row["OnetCode"]
    }
    url_params = "&".join([f"{key}={value}" for key, value in parameters.items()])
    url = f'{base_url}{url_params}'
    return f'<a href="{url}">{row["Career"]}</a>'

## Email Configuration:

In [5]:
# Define constants
SENDER_EMAIL = "kamran.techimpact@gmail.com"
RECEIVER_EMAIL = "kamran.makarian@gmail.com"
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_PASSWORD = password

## Email Sending Logic:

In [6]:
class ApiClient:
    apiUri = 'https://api.elasticemail.com/v2'

    def __init__(self, api_key):
        self.apiKey = api_key

    def Request(self, method, url, data):
        data['apikey'] = self.apiKey
        if method == 'POST':
            result = requests.post(f'{self.apiUri}{url}', data=data)
        elif method == 'PUT':
            result = requests.put(f'{self.apiUri}{url}', data=data)
        elif method == 'GET':
            result = requests.get(f'{self.apiUri}{url}', params=data)

        jsonMy = result.json()

        if jsonMy['success'] is False:
            return jsonMy['error']

        return jsonMy['data']

def Send(subject, EEfrom, fromName, to, bodyHtml, bodyText, isTransactional, api_key):
    client = ApiClient(api_key)
    return client.Request('POST', '/email/send', {
        'subject': subject,
        'from': EEfrom,
        'fromName': fromName,
        'to': to,
        'bodyHtml': bodyHtml,
        'bodyText': bodyText,
        'isTransactional': isTransactional
    })

## Main Execution:

In [7]:
# Load data
data = load_data('/Users/kamran/Downloads/DWDB/repo/career-onestop/data/skills-submit-api/example-response.json')

# Extract and format data
key_list = list(data.keys())
inner_dict0 = data[key_list[0]][:20]
recoms_df = pd.DataFrame(inner_dict0)
recoms_df_modified = recoms_df[['Rank', 
                                'OccupationTitle', 
                                'AnnualWages', 
                                'TypicalEducation', 
                                'Outlook', 
                                'OnetCode']].copy()

recoms_df_modified.rename(columns={'Rank': 'Your Match', 
                                   'OccupationTitle': 'Career', 
                                   'AnnualWages': 'Annual Wages', 
                                   'TypicalEducation': 'Education'}, inplace=True)

# Embed hyperlinks in Career column
recoms_df_modified["Career"] = recoms_df_modified.apply(create_hyperlink, axis=1)
# Format Annual Wages
recoms_df_modified['Annual Wages'] = recoms_df_modified['Annual Wages'].apply(format_annual_wages)

# Drop redundant columns
recoms_df_modified.drop('OnetCode', axis=1, inplace=True)

column_alignments = {
    "Your Match": "center",
    "Career": "left",
    "Annual Wages": "right",
    "Education": "left",
    "Outlook": "left"
}

# Extract the alignment values into a list in the same order as columns
alignment_list = [column_alignments.get(column, "left") for column in recoms_df_modified.columns]

# Tabulate the info with specified column alignments
table = tabulate(
    recoms_df_modified.to_dict(orient='records'),
    headers="keys",
    tablefmt="html",
    colalign=alignment_list
)

# Find the current date
current_date = datetime.now().strftime("%B %d, %Y")

# Create subject and body of the message
subject = f'Your survey result for {current_date}'

In [8]:
# Style the table headers and cells
table_headers = recoms_df_modified.columns.tolist()
table_html = "<table>"
table_html += "<tr>"

# Style for table headers
header_style = "font-weight: bold; font-size: 20px;"

# Style for table cells
cell_style = "font-weight: normal; font-size: 16px;"  # You can change the font size as needed

for header in table_headers:
    table_html += f"<th style='{header_style}'>{header}</th>"
table_html += "</tr>"

# Cell styling
for record in recoms_df_modified.to_dict(orient='records'):
    table_html += "<tr>"
    for header in table_headers:
        table_html += f"<td style='{cell_style}'>{record[header]}</td>"
    table_html += "</tr>"
table_html += "</table>"

# Style for the message
message_style = "font-weight: normal; font-size: 16px;"

# Use the table_html in your email body
message = f'<div style="{message_style}">Your customized top 20 suggestion list is as follows:</div>\n\n{table_html}'

result = Send(subject,
              SENDER_EMAIL,
              "Tech Impact",
              RECEIVER_EMAIL,
              f"<h1>{message}</h1>",
              f"{message}",
              True,
              api_key)

print(result)

{'transactionid': 'c517270a-d82c-01a9-c00a-542d3f9401da', 'messageid': 'NBzfIIZ-pNjYSsz3jgC6bQ2'}


# The End